<center><h3>Predict The Price of Books</h3></center>

### Load all libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder as le ,MinMaxScaler, RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import math
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV, cross_val_score, KFold, learning_curve
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

## Load Dataset

In [2]:
train = pd.read_excel(r"C:\Users\ashok\Data\Participants_Data\Data_Train.xlsx")
train.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


In [3]:
test = pd.read_excel(r"C:\Users\ashok\Data\Participants_Data\Data_Test.xlsx")
test.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
0,The Complete Sherlock Holmes: 2 Boxes sets,Sir Arthur Conan Doyle,"Mass Market Paperback,– 1 Oct 1986",4.4 out of 5 stars,960 customer reviews,A collection of entire body of work of the She...,Short Stories (Books),"Crime, Thriller & Mystery"
1,Learn Docker - Fundamentals of Docker 18.x: Ev...,Gabriel N. Schenker,"Paperback,– Import, 26 Apr 2018",5.0 out of 5 stars,1 customer review,Enhance your software deployment workflow usin...,Operating Systems Textbooks,"Computing, Internet & Digital Media"
2,Big Girl,Danielle Steel,"Paperback,– 17 Mar 2011",5.0 out of 5 stars,4 customer reviews,"'Watch out, world. Here I come!'\nFor Victoria...",Romance (Books),Romance
3,Think Python: How to Think Like a Computer Sci...,Allen B. Downey,"Paperback,– 2016",4.1 out of 5 stars,11 customer reviews,"If you want to learn how to program, working w...",Programming & Software Development (Books),"Computing, Internet & Digital Media"
4,Oxford Word Skills: Advanced - Idioms & Phrasa...,Redman Gairns,"Paperback,– 26 Dec 2011",4.4 out of 5 stars,9 customer reviews,"Learn and practise the verbs, prepositions and...",Linguistics (Books),"Language, Linguistics & Writing"


In [4]:
print('Train data contains', train.shape[0], 'rows and', train.shape[1], 'columns')
print('Test data contains', test.shape[0], 'rows and', test.shape[1], 'columns')

Train data contains 6237 rows and 9 columns
Test data contains 1560 rows and 8 columns


## Merging both the datasets for EDA

In [5]:
df = pd.concat([test.assign(ind="test"), train.assign(ind="train")])

In [6]:
df

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,ind,Price
0,The Complete Sherlock Holmes: 2 Boxes sets,Sir Arthur Conan Doyle,"Mass Market Paperback,– 1 Oct 1986",4.4 out of 5 stars,960 customer reviews,A collection of entire body of work of the She...,Short Stories (Books),"Crime, Thriller & Mystery",test,NaN
1,Learn Docker - Fundamentals of Docker 18.x: Ev...,Gabriel N. Schenker,"Paperback,– Import, 26 Apr 2018",5.0 out of 5 stars,1 customer review,Enhance your software deployment workflow usin...,Operating Systems Textbooks,"Computing, Internet & Digital Media",test,NaN
2,Big Girl,Danielle Steel,"Paperback,– 17 Mar 2011",5.0 out of 5 stars,4 customer reviews,"'Watch out, world. Here I come!'\nFor Victoria...",Romance (Books),Romance,test,NaN
3,Think Python: How to Think Like a Computer Sci...,Allen B. Downey,"Paperback,– 2016",4.1 out of 5 stars,11 customer reviews,"If you want to learn how to program, working w...",Programming & Software Development (Books),"Computing, Internet & Digital Media",test,NaN
4,Oxford Word Skills: Advanced - Idioms & Phrasa...,Redman Gairns,"Paperback,– 26 Dec 2011",4.4 out of 5 stars,9 customer reviews,"Learn and practise the verbs, prepositions and...",Linguistics (Books),"Language, Linguistics & Writing",test,NaN
...,...,...,...,...,...,...,...,...,...,...
6232,Humans: A Brief History of How We F*cked It Al...,Tom Phillips,"Paperback,– 8 Aug 2018",5.0 out of 5 stars,2 customer reviews,'F*cking brilliant' Sarah Knight\n'Very funny'...,Anthropology (Books),Humour,train,322.0
6233,The Chemist,Stephenie Meyer,"Paperback,– 21 Nov 2016",3.3 out of 5 stars,9 customer reviews,"In this gripping page-turner, an ex-agent on t...",Contemporary Fiction (Books),"Crime, Thriller & Mystery",train,421.0
6234,The Duke And I: Number 1 in series (Bridgerton...,Julia Quinn,"Paperback,– 8 Jun 2006",3.8 out of 5 stars,3 customer reviews,'The most refreshing and radiant love story yo...,Romance (Books),Romance,train,399.0
6235,Frostfire (Kanin Chronicles),Amanda Hocking,"Paperback,– 15 Jan 2015",3.5 out of 5 stars,4 customer reviews,Frostfire by Amanda Hocking is the stunning fi...,Action & Adventure (Books),Action & Adventure,train,319.0


## Basic Dataset information

In [7]:
df.shape

(7797, 10)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7797 entries, 0 to 6236
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         7797 non-null   object 
 1   Author        7797 non-null   object 
 2   Edition       7797 non-null   object 
 3   Reviews       7797 non-null   object 
 4   Ratings       7797 non-null   object 
 5   Synopsis      7797 non-null   object 
 6   Genre         7797 non-null   object 
 7   BookCategory  7797 non-null   object 
 8   ind           7797 non-null   object 
 9   Price         6237 non-null   float64
dtypes: float64(1), object(9)
memory usage: 670.1+ KB


In [9]:
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
print(cat_cols)

num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
print(num_cols)

['Title', 'Author', 'Edition', 'Reviews', 'Ratings', 'Synopsis', 'Genre', 'BookCategory', 'ind']
['Price']


In [10]:
df.describe(percentiles=[0.01, 0.02,0.03, 0.04, 0.05, 0.1, .25, .5, .75, .9, .95, .96, .97, .98, .99])

,Price
count,6237.000000
mean,560.707516
std,690.110657
min,25.000000
1%,68.360000
2%,90.000000
3%,100.000000
4%,114.000000
5%,123.000000
10%,166.252000


In [11]:
df.describe(include=object) 

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,ind
count,7797,7797,7797,7797,7797,7797,7797,7797,7797
unique,6787,4372,3882,36,368,6771,363,11,2
top,"Innovators: How a Group of Inventors, Hackers,...",Agatha Christie,"Paperback,– 5 Oct 2017",5.0 out of 5 stars,1 customer review,A Tinkle Double Digest is two Tinkle Digests i...,Action & Adventure (Books),Action & Adventure,train
freq,4,87,60,1751,1328,8,1183,1036,6237


## Converting the data in Lower case

In [12]:
def to_lower(df,cols):
    for col in cols:
        df[col] = df[col].str.lower()

In [13]:
cols = ['Title','Author','Edition','Synopsis','Genre','BookCategory']

to_lower(df,cols)

## Null Value Treatment

In [14]:
df.isnull().sum()

Title              0
Author             0
Edition            0
Reviews            0
Ratings            0
Synopsis           0
Genre              0
BookCategory       0
ind                0
Price           1560
dtype: int64

## Outlier Treatment

In [15]:
df["Price"]=np.where(df["Price"]>3403.360000, 3403.360000, df["Price"])

In [16]:
df.describe()

,Price
count,6237.000000
mean,542.061414
std,533.592329
min,25.000000
25%,249.180000
50%,373.000000
75%,599.000000
max,3403.360000


## EDA

In [17]:
# extrating only the review from the "Reviews" column
df['Reviews'] = df.Reviews.apply(lambda r: float(r.split()[0]))

In [18]:
# extracting only ratings from the "Rating" column
df['Ratings']= df.Ratings.str.extract('(\d+)')
# changing the data type of Ratings column
df["Ratings"] = df.Ratings.astype(float)

In [19]:
# removing words books and textbooks from columns "Genre"
df.Genre = df.Genre.apply(lambda x: x.strip('(books)').strip('Textbooks'))

In [20]:
# 'Extracting' year of publish from the `Edition` column
df['Edition_Year'] = df['Edition'].apply(lambda x: x.split()[-1] if x.split()[-1].isdigit() else 'na')

In [21]:
df.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,ind,Price,Edition_Year
0,the complete sherlock holmes: 2 boxes sets,sir arthur conan doyle,"mass market paperback,– 1 oct 1986",4.4,960.0,a collection of entire body of work of the she...,hort stories,"crime, thriller & mystery",test,NaN,1986
1,learn docker - fundamentals of docker 18.x: ev...,gabriel n. schenker,"paperback,– import, 26 apr 2018",5.0,1.0,enhance your software deployment workflow usin...,perating systems,"computing, internet & digital media",test,NaN,2018
2,big girl,danielle steel,"paperback,– 17 mar 2011",5.0,4.0,"'watch out, world. here i come!'\nfor victoria...",romance,romance,test,NaN,2011
3,think python: how to think like a computer sci...,allen b. downey,"paperback,– 2016",4.1,11.0,"if you want to learn how to program, working w...",programming & software development,"computing, internet & digital media",test,NaN,2016
4,oxford word skills: advanced - idioms & phrasa...,redman gairns,"paperback,– 26 dec 2011",4.4,9.0,"learn and practise the verbs, prepositions and...",linguistics,"language, linguistics & writing",test,NaN,2011


In [22]:
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
print(cat_cols)

num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
print(num_cols)

['Title', 'Author', 'Edition', 'Synopsis', 'Genre', 'BookCategory', 'ind', 'Edition_Year']
['Reviews', 'Ratings', 'Price']


## Applying PowerTransformation

In [23]:
from sklearn.preprocessing import PowerTransformer

In [24]:
power = PowerTransformer()

In [25]:
df[['Reviews']] = power.fit_transform(df[['Reviews']])
df[['Ratings']] = power.fit_transform(df[['Ratings']])
df[['Price']] = power.fit_transform(df[['Price']])

In [26]:
df.describe(include="object")

,Title,Author,Edition,Synopsis,Genre,BookCategory,ind,Edition_Year
count,7797,7797,7797,7797,7797,7797,7797,7797
unique,6782,4359,3882,6770,359,11,2,59
top,"innovators: how a group of inventors, hackers,...",agatha christie,"paperback,– 5 oct 2017",a tinkle double digest is two tinkle digests i...,action & adventure,action & adventure,train,2018
freq,4,87,60,8,1183,1036,6237,1018


### Since, it is impossible to make dummies of columns with so many unique values, we will remove all the columns which are not required for builing the model and will only create dummy for "Book_Category"

In [27]:
df.drop(columns=['Edition','Synopsis','Title'],inplace=True)
df

,Author,Reviews,Ratings,Genre,BookCategory,ind,Price,Edition_Year
0,sir arthur conan doyle,-0.011370,2.225157,hort stories,"crime, thriller & mystery",test,NaN,1986
1,gabriel n. schenker,1.350579,-1.414916,perating systems,"computing, internet & digital media",test,NaN,2018
2,danielle steel,1.350579,-0.370980,romance,romance,test,NaN,2011
3,allen b. downey,-0.537118,0.388788,programming & software development,"computing, internet & digital media",test,NaN,2016
4,redman gairns,-0.011370,0.246759,linguistics,"language, linguistics & writing",test,NaN,2011
...,...,...,...,...,...,...,...,...
6232,tom phillips,1.350579,-0.917026,anthropology,humour,train,-0.245632,2018
6233,stephenie meyer,-1.536830,0.246759,contemporary fiction,"crime, thriller & mystery",train,0.115748,2016
6234,julia quinn,-0.974748,-0.599135,romance,romance,train,0.044242,2006
6235,amanda hocking,-1.334879,-0.370980,action & adventure,action & adventure,train,-0.258442,2015


In [28]:
df = df.astype({'Author':'category','Genre':'category','BookCategory':'category','Edition_Year':'category'})

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7797 entries, 0 to 6236
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Author        7797 non-null   category
 1   Reviews       7797 non-null   float64 
 2   Ratings       7797 non-null   float64 
 3   Genre         7797 non-null   category
 4   BookCategory  7797 non-null   category
 5   ind           7797 non-null   object  
 6   Price         6237 non-null   float64 
 7   Edition_Year  7797 non-null   category
dtypes: category(4), float64(3), object(1)
memory usage: 527.2+ KB


## Label Encoding

In [30]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [31]:
cols_toEncode = ['Author','Genre','BookCategory','Edition_Year']

In [32]:
df['Author'] = le.fit_transform(df['Author'])

In [33]:
df['Genre'] = le.fit_transform(df['Genre'])

In [34]:
df['BookCategory'] = le.fit_transform(df['BookCategory'])

In [35]:
df['Edition_Year'] = le.fit_transform(df['Edition_Year'])

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7797 entries, 0 to 6236
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Author        7797 non-null   int32  
 1   Reviews       7797 non-null   float64
 2   Ratings       7797 non-null   float64
 3   Genre         7797 non-null   int32  
 4   BookCategory  7797 non-null   int32  
 5   ind           7797 non-null   object 
 6   Price         6237 non-null   float64
 7   Edition_Year  7797 non-null   int32  
dtypes: float64(3), int32(4), object(1)
memory usage: 426.4+ KB


## Train-Test Split

In [37]:
test, train = df[df["ind"].eq("test")], df[df["ind"].eq("train")]

In [38]:
train.drop("ind", axis=1, inplace=True)

In [39]:
test.drop("ind", axis=1, inplace=True)

In [40]:
train.shape

(6237, 7)

In [41]:
test.drop("Price", axis=1, inplace=True)

In [42]:
test.shape

(1560, 6)

## Model Building - LightGBM

LightGBM is a gradient boosting framework that uses tree based learning algorithms. It is designed to be distributed and efficient with the following advantages:

    - Faster training speed and higher efficiency.
    - Lower memory usage.
    - Better accuracy.
    - Support of parallel and GPU learning.
    - Capable of handling large-scale data.

In [43]:
y_train = train.pop('Price')
X_train = train

In [44]:
import lightgbm as lgb

In [45]:
#convert the dataset to lgbDataset
df_train = lgb.Dataset(X_train,y_train)

In [46]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l1', 'auc'],
    'learning_rate': 1,
    'categorical_feature':[0,3,4,5],
    'num_iterations':10000
}

In [47]:
rg = lgb.LGBMRegressor(**hyper_params)

In [48]:
rg.fit(X_train,y_train)

LGBMRegressor(categorical_feature=[0, 3, 4, 5], learning_rate=1,
              metric=['l1', 'auc'], num_iterations=10000,
              objective='regression', task='train')

In [49]:
y_pred_train = rg.predict(X_train)

In [50]:
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score

In [51]:
y_pred_train

array([-0.77754367, -0.89314967, -0.34740639, ...,  0.0442417 ,
       -0.2584422 , -0.39300204])

In [52]:
print("Train MSE:",np.sqrt(mean_squared_error(y_train,y_pred_train))) 

Train MSE: 0.17046944357729923


In [53]:
print("Train r2_score:",r2_score(y_train, y_pred_train))

Train r2_score: 0.970940168806446


In [54]:
y_test_pred = rg.predict(test)

In [55]:
y_test_pred = pd.DataFrame(y_test_pred,columns=['Price'])

In [56]:
predictions = power.inverse_transform(y_test_pred[['Price']])

In [57]:
pd.DataFrame(predictions,columns=['Price']).to_csv('my_submission_1.csv',index=None)